In [10]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import lightgbm as lgb
import catboost as cb

from tsfresh.examples import load_robot_execution_failures
from tsfresh import extract_features, select_features
import optuna

from common import EP
from dfdb import DFDB

import numpy as np
from sklearn.decomposition import PCA

import types
import copy





In [11]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [12]:
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_colwidth', -1)

In [13]:
df_train = pd.read_pickle('../feats/df_train.pkl')
df_test = pd.read_pickle('../feats/df_test.pkl')

df_train['label'] = df_train['y'].apply(lambda x:  int(x) if x<15 else 15)
group = df_train['season'].values
group[np.where(group==17)[0]] = 1
df_train['group'] = group
df_train = df_train.drop(columns=['season'])

In [5]:
# db_lgbm = DFDB('../trial/lgbm.pkl', auto_commit=False)
# df_trial_lgbm = db_lgbm.select()
# df_trial_lgbm['kfold'] = df_trial_lgbm['param'].apply(lambda x: x['kfold'])
# df_trial_lgbm['kfold-type'] = df_trial_lgbm['param'].apply(lambda x: x['kfold']['type'])

In [6]:
# df_trial_lgbm[(df_trial_lgbm['kfold-type']=='group')&(df_trial_lgbm['mae_diff']<.2)].sort_values(by=['val_mae'])[['datetime','nfeatures','kfold-type','train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']].head()

,datetime,nfeatures,kfold-type,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
2498,2019-05-23 23:32:01.229662,14,group,1.882727,0.010383,2.058523,0.488644,0.175796
2546,2019-05-24 00:04:53.793038,14,group,1.872995,0.010356,2.060622,0.477251,0.187627
2515,2019-05-23 23:41:56.647154,14,group,1.867831,0.010190,2.060701,0.487186,0.192870
2473,2019-05-23 23:16:18.127971,14,group,1.889906,0.010449,2.060748,0.482239,0.170842
2367,2019-05-23 21:45:02.439202,14,group,1.891329,0.010296,2.061111,0.478845,0.169782


In [14]:
df_spec_train = pd.read_pickle('../feats/spec_features.pkl')
df_spec_test = pd.read_pickle('../feats/spec_features_test.pkl')

In [15]:
df_train = pd.merge(df_train, df_spec_train, on='index')
df_test = pd.merge(df_test, df_spec_test, on='index')

In [16]:
test_X = df_train.drop(columns=['y','index','group','label']).copy()
test_X.index = df_train['index']
test_y = df_train['y'].copy()
test_y.index = df_train['index']
tsfresh_columns = select_features(test_X, test_y).columns.tolist()

In [17]:
len(tsfresh_columns)

9731

In [18]:
db = DFDB('../trial/lgbm4.pkl', auto_commit=False)

In [19]:
df_trial = db.select()

In [12]:
# group = EP.bubble_merge(df_train, columns=tsfresh_columns, base_correlation_coefficient=.9, coverage_rate=.9)

In [69]:
param822 = {'columns': tsfresh_columns,
 'kfold': {'n_splits': 4,
  'random_state': 1985,
  'shuffle': True,
  'type': 'group'},
 'scaler': {'cls': 'StandardScaler', 'init':{}},
 'algorithm': {'cls': 'lgb.LGBMRegressor',
  'init': {'learning_rate': 0.20562566891912665,
   'feature_fraction': 0.6442827431025538,
   'bagging_fraction': 0.802261802916953,
   'min_data_in_leaf': 704,
   'lambda_l1': 65.43072344708864,
   'lambda_l2': 62.4903783326351,
   'max_bin': 96,
   'num_leaves': 4,
   'random_state': 5943,
   'n_jobs': 16},
  'fit': {}},
 }

In [28]:
mytrial =[]
df_his, df_feature_importances, df_valid_pred, df_test_pred = EP.process(df_train, param822, df_test=df_test, trial=mytrial, is_output_feature_importance=True, remark='init_try')

In [30]:
df_trial = pd.DataFrame(mytrial)

In [33]:
df_trial[['datetime','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
0,2019-05-30 06:03:08.407859,9731,1.833261,0.002514,2.206499,0.018641,0.373238


In [31]:
df_feature_importances = df_trial['df_feature_importances'].values[0]
top200 = EP.evaluate(df_feature_importances)[:200]

In [9]:
top200 =['spkt_welch_densitycoeff_2',
 'spkt_welch_density__coeff_3',
 'q25_roll_std_100',
 '3th_peak_freq',
 'min_roll_std_100',
 'q05_roll_std_100',
 'iqr_6',
 'abs_max_8',
 'mfcc_5_abs_mean',
 "number_peaks{'n': 10}",
 'ave10_7',
 'mfcc_13_mean',
 "autocorrelation{'lag': 5}",
 'mfcc_accelerate_8_variance',
 'abs_max_7',
 'mfcc_accelerate_1_kurtosis',
 'q05_roll_std_1000',
 'spkt_welch_density__coeff_42',
 'max_to_min_diff_5',
 'mfcc_13_quantile25',
 'fft_coefficientcoeff_80__attr_"imag"',
 'abs_q25_5',
 'mfcc_5_mean',
 'median__roll_std',
 '5000skewness_max_',
 'fft_coefficientcoeff_6__attr_"abs"',
 'partial_autocorrelationlag_5',
 'abs_min_8',
 'spkt_welch_density__coeff_28',
 'ar_coefficientk_10__coeff_3',
 'abs_q75_7',
 'mfcc_accelerate_15_min',
 'abs_max_4',
 'mfcc_10_quantile25',
 "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
 'spkt_welch_density__coeff_79',
 "value_count{'value': 1}",
 '5000peak_peak_amp_max_',
 'min__roll_std',
 'mfcc_4_rolling_std_mean',
 'q01_roll_std_1000',
 'mfcc_3_abs_q75',
 'mfcc_5_quantile95',
 'fft_coefficientcoeff_16__attr_"imag"',
 'abs_q01_4',
 'mfcc_6_quantile05',
 'agg_autocorrelationf_agg_"mean"__maxlag_40',
 '5000quantile25skewness_',
 'mfcc_0_abs_q25',
 'mfcc_delta_3_quantile01',
 'spkt_welch_density__coeff_99',
 'mfcc_9_mean',
 'mfcc_12_mean',
 'q75_roll_mean_10',
 'max_to_min',
 'mfcc_3_abs_mean',
 'kurt_7',
 'mfcc_14_quantile01',
 'mfcc_10_mean',
 'mfcc_delta_13_min',
 'mfcc_delta_13_abs_max',
 '5000no_zero_crossing_mean_',
 'abs_q75_6',
 'mfcc_12_quantile99',
 'mfcc_accelerate_6_skewness',
 'mfcc_5_max',
 '5000smoothness_entropy_',
 '5000median_skewness_',
 'mfcc_accelerate_8_min',
 'abs_min_3',
 '5000quantile75mean_',
 "number_crossing_m{'m': 1}",
 '5000smoothness_std_',
 'mfcc_9_abs_q25',
 'flac3_1_quantile99',
 'mfcc_5_skewness',
 'fft_coefficientcoeff_56__attr_"angle"',
 'fft_coefficientcoeff_70__attr_"abs"',
 'mfcc_10_abs_q75',
 'fft_coefficientcoeff_24__attr_"angle"',
 'med_7',
 'spkt_welch_density__coeff_73',
 'abs_q99_8',
 'ave10_6',
 'spkt_welch_density__coeff_38',
 'skew_1',
 'mfcc_delta_3_abs_q95',
 "change_quantiles{'ql': 0.6, 'qh': 1.0, 'isabs': False, 'f_agg': 'mean'}",
 'max_to_min_5',
 'mfcc_delta_4_quantile75',
 'abs_max_1',
 '5000crest_factor_quantile75',
 'partial_autocorrelationlag_1',
 'mfcc_delta_11_quantile01',
 'mfcc_accelerate_7_quantile01',
 'mfcc_8_rolling_std_mean',
 'q95_roll_mean_100',
 'mfcc_delta_6_rolling_std_mean',
 'flac3_0_min',
 'fft_coefficientcoeff_36__attr_"abs"',
 'iqr_8',
 'max_9',
 '5000smoothness_quantile05',
 'mfcc_1_kurtosis',
 'mfcc_7_abs_q95',
 'spkt_welch_density__coeff_66',
 'spkt_welch_density__coeff_64',
 'mfcc_3_abs_q95',
 '5000skewness_mean_',
 'mfcc_10_abs_mean',
 '5000quantile99quantile01',
 'mean_change_rate',
 'mfcc_accelerate_5_rolling_std_mean',
 'mfcc_accelerate_5_quantile05',
 '5000rms_median_',
 'flac3_1_abs_mean',
 '5000rms_quantile05',
 '5000quantile75quantile25',
 'mfcc_accelerate_12_min',
 "value_count{'value': -1}",
 'mfcc_12_abs_max',
 'abs_min_5',
 'mfcc_5_abs_q99',
 'mfcc_delta_5_quantile01',
 'mfcc_12_quantile75',
 'fft_coefficientcoeff_56__attr_"imag"',
 'spkt_welch_densitycoeff_5',
 'mfcc_0_abs_q05',
 'mfcc_13_quantile75',
 'mfcc_delta_5_min',
 'mfcc_5_quantile99',
 'fft_coefficientcoeff_8__attr_"angle"',
 'spkt_welch_density__coeff_30',
 'mfcc_accelerate_4_max',
 'mfcc_14_median',
 "change_quantiles{'ql': 0.2, 'qh': 0.4, 'isabs': False, 'f_agg': 'var'}",
 'mfcc_5_median',
 'mfcc_accelerate_5_abs_q75',
 'spkt_welch_density__coeff_59',
 'mfcc_accelerate_5_kurtosis',
 'mfcc_delta_9_quantile99',
 'mfcc_5_quantile25',
 'spkt_welch_density__coeff_113',
 'mfcc_2_quantile05',
 'spkt_welch_density__coeff_58',
 'mfcc_5_abs_q75',
 'spkt_welch_density__coeff_22',
 'spkt_welch_density__coeff_115',
 'spkt_welch_density__coeff_4',
 'mfcc_5_abs_q05',
 'spkt_welch_density__coeff_25',
 'mfcc_5_abs_max',
 'spkt_welch_density__coeff_27',
 'mfcc_4_quantile75',
 'mfcc_3_rolling_std_mean',
 'mfcc_4_median',
 'mfcc_0_quantile01',
 '5000quantile75rssq_',
 'fft_coefficientcoeff_24__attr_"imag"',
 'mfcc_delta_12_kurtosis',
 '5000rms_quantile25',
 'mfcc_13_quantile05',
 'fft_coefficientcoeff_8__attr_"imag"',
 'mfcc_delta_11_rolling_std_mean',
 'mfcc_delta_12_quantile99',
 'mfcc_accelerate_9_abs_q25',
 'mfcc_11_abs_mean',
 'mfcc_delta_15_abs_max',
 'abs_max_2',
 'mfcc_11_abs_std',
 'abs_max_roll_mean_1000',
 'peak_to_average_power_ratio__roll_mean',
 'mfcc_13_median',
 'fft_coefficientcoeff_62__attr_"abs"',
 'mfcc_11_quantile75',
 "quantile{'q': 0.8}",
 'mfcc_delta_3_rolling_std_mean',
 'abs_q99_7',
 'mfcc_delta_12_max',
 'mfcc_12_min',
 'mfcc_10_quantile99',
 'mfcc_14_abs_q99',
 'mfcc_14_quantile05',
 'mfcc_14_quantile25',
 'kurt_1',
 'mfcc_2_median',
 'q01_2',
 'mfcc_2_abs_q25',
 'kurt_8',
 'mfcc_accelerate_6_abs_max',
 'mfcc_1_quantile75',
 'q05_5',
 'abs_q95_9',
 'q05_roll_mean_100',
 'mfcc_10_abs_q25',
 'q75_9',
 'mfcc_15_mean',
 "change_quantiles{'ql': 0.6, 'qh': 0.8, 'isabs': True, 'f_agg': 'mean'}",
 'flac3_0_abs_std',
 "change_quantiles{'ql': 0.4, 'qh': 1.0, 'isabs': False, 'f_agg': 'mean'}"]

In [10]:
param822 = {'columns': top200,
 'kfold': {'n_splits': 3,
  'random_state': 1985,
  'shuffle': True,
  'type': 'group'},
 'scaler': {'cls': 'StandardScaler', 'init':{}},
 'algorithm': {'cls': 'lgb.LGBMRegressor',
  'init': {'learning_rate': 0.20562566891912665,
   'feature_fraction': 0.6442827431025538,
   'bagging_fraction': 0.802261802916953,
   'min_data_in_leaf': 704,
   'lambda_l1': 65.43072344708864,
   'lambda_l2': 62.4903783326351,
   'max_bin': 96,
   'num_leaves': 4,
   'random_state': 5943,
   'n_jobs': 16},
  'fit': {}},
 }

In [77]:
mytrial =[]
EP.select_features_(df_train, param822, mytrial, nfeats_best=10, nfeats_removed_per_try=1, key='average_permutation_weight', remark='rfe to 10 group3')

In [26]:
mytrial =[]
EP.select_features_(df_train, param822, mytrial, nfeats_best=20, nfeats_removed_per_try=10, key='average_model_weight', remark='rfe to 10 group3 model weight')

In [15]:
for trial_i in mytrial:
    db.insert(trial_i)
df_trial = db.select()

In [29]:
df_trial[df_trial['remark']=='rfe to 10 group3'][['datetime','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff', 'remark']]

,datetime,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff,remark
200,2019-05-30 07:44:54.376348,200,1.833266,0.001247,2.114163,0.007348,0.280897,rfe to 10 group3
201,2019-05-30 07:46:21.760008,190,1.836945,0.001168,2.105274,0.006524,0.268329,rfe to 10 group3
202,2019-05-30 07:47:46.915482,180,1.843670,0.001261,2.090790,0.005840,0.247120,rfe to 10 group3
203,2019-05-30 07:49:03.180761,170,1.863349,0.001791,2.075974,0.009361,0.212624,rfe to 10 group3
204,2019-05-30 07:50:18.579707,160,1.868806,0.001823,2.054681,0.011789,0.185875,rfe to 10 group3
205,2019-05-30 07:51:31.811278,150,1.874641,0.002162,2.059701,0.012927,0.185060,rfe to 10 group3
206,2019-05-30 07:52:40.306392,140,1.876497,0.001987,2.055771,0.011990,0.179274,rfe to 10 group3
207,2019-05-30 07:53:44.131969,130,1.878320,0.002075,2.045999,0.012080,0.167680,rfe to 10 group3
208,2019-05-30 07:54:46.528676,120,1.880426,0.002166,2.047209,0.012268,0.166783,rfe to 10 group3
209,2019-05-30 07:55:42.315509,110,1.886064,0.002286,2.046174,0.012501,0.160110,rfe to 10 group3


In [31]:
score = df_trial.loc[217].val_mae

In [32]:
param = df_trial.loc[217]['param']

In [40]:
def width_frist_rfe(df_train, param, trial, score, df_test=None, remark=None):

    param_ = copy.deepcopy(param)
    columns_ = param_['columns']
    best_score = score
    best_param = param_
    for col in columns_:
        param_['columns'] = list(set(columns_) - set([col]))
        df_his, df_feature_importances, df_valid_pred, df_test_pred = EP.process(df_train, param_, df_test=df_test, trial=trial, is_output_feature_importance=False, remark=remark)
        val_mae_mean = np.mean(df_his.valid)
        if val_mae_mean<best_score:
            best_score = val_mae_mean
            best_param = copy.deepcopy(param_)

    if best_score < score:
        width_frist_rfe(df_train, best_param, trial, best_score, df_test, remark=remark)

    return

In [41]:
mytrial=[]
width_frist_rfe(df_train, param, mytrial, score, df_test=df_test, remark='wf 217')

In [43]:
for trial_i in mytrial:
    db.insert(trial_i)
df_trial = db.select()

In [44]:
df_trial[(df_trial['remark']=='wf 217')].sort_values(by=['val_mae'])[['datetime','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
615,2019-05-31 10:42:57.154150,26,1.924369,0.002107,2.038264,0.012776,0.113895
600,2019-05-31 10:38:04.907831,27,1.921733,0.002079,2.039145,0.014210,0.117412
616,2019-05-31 10:43:15.163221,26,1.922316,0.002172,2.039687,0.014872,0.117372
629,2019-05-31 10:49:13.308707,25,1.926718,0.002177,2.040725,0.012472,0.114006
621,2019-05-31 10:45:17.758022,26,1.922198,0.002096,2.040836,0.013688,0.118638
624,2019-05-31 10:46:47.740361,26,1.926255,0.002166,2.040850,0.012878,0.114595
642,2019-05-31 10:55:36.962151,25,1.923983,0.002091,2.041372,0.013562,0.117389
652,2019-05-31 11:00:34.663968,25,1.922588,0.002183,2.041373,0.013508,0.118785
635,2019-05-31 10:52:10.458742,25,1.924030,0.002238,2.041382,0.013515,0.117352
628,2019-05-31 10:48:44.072253,25,1.926393,0.002261,2.041794,0.012991,0.115401


In [20]:
param=df_trial.loc[615]['param']

In [11]:
param = {'columns': ['mfcc_10_abs_q75',
  'q25_roll_std_100',
  'iqr_6',
  'mfcc_9_mean',
  'abs_q75_7',
  'mfcc_delta_5_min',
  'mfcc_delta_3_quantile01',
  'abs_q75_6',
  'q05_roll_std_100',
  'mfcc_accelerate_1_kurtosis',
  'mfcc_5_mean',
  'spkt_welch_density__coeff_42',
  'mfcc_3_rolling_std_mean',
  'mfcc_12_mean',
  '5000smoothness_entropy_',
  'mfcc_13_mean',
  "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
  'mfcc_5_abs_max',
  'abs_q25_5',
  'mfcc_5_quantile95',
  'spkt_welch_densitycoeff_5',
  '5000crest_factor_quantile75',
  'spkt_welch_densitycoeff_2',
  'mfcc_accelerate_8_variance',
  "number_peaks{'n': 10}",
  'spkt_welch_density__coeff_3'],
 'kfold': {'n_splits': 3,
  'random_state': 1985,
  'shuffle': True,
  'type': 'group'},
 'scaler': {'cls': 'StandardScaler', 'init': {}},
 'algorithm': {'cls': 'lgb.LGBMRegressor',
  'init': {'learning_rate': 0.20562566891912665,
   'feature_fraction': 0.6442827431025538,
   'bagging_fraction': 0.802261802916953,
   'min_data_in_leaf': 704,
   'lambda_l1': 65.43072344708864,
   'lambda_l2': 62.4903783326351,
   'max_bin': 96,
   'num_leaves': 4,
   'random_state': 5943,
   'n_jobs': 16},
  'fit': {}}}

In [24]:
mytrial = []
#  tune hypterparameters
def objective(trial):
        
    learning_rate = trial.suggest_uniform('learning_rate', .01, .5)
    feature_fraction = trial.suggest_uniform('feature_fraction', .6, 1)
    bagging_fraction = trial.suggest_uniform('bagging_fraction', 0.6, 1)
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 200, 800)
    lambda_l1 = trial.suggest_loguniform('lambda_l1', 1e-6, 1e2)
    lambda_l2 = trial.suggest_loguniform('lambda_l2', 1e-6, 1e2)
    max_bin = trial.suggest_int('max_bin', 10, 100)
    num_leaves = trial.suggest_int('num_leaves', 4, 32)
    random_state = trial.suggest_int('random_state', 1, 9999)
        
    args={
        'columns':df_trial.loc[615]['param']['columns'],
        'kfold':{
            'n_splits': 3,
            'random_state': 1985,
            'shuffle': True,
            'type': 'stratified'
        },
        'scaler':{
            'cls':'StandardScaler',
            'init':{}
        },
        'algorithm':{
            'cls':'lgb.LGBMRegressor',
            'init':{
                'learning_rate':learning_rate,
                'feature_fraction':feature_fraction,
                'bagging_fraction':bagging_fraction,
                'min_data_in_leaf':min_data_in_leaf,
                'lambda_l1':lambda_l1,
                'lambda_l2':lambda_l2,
                'max_bin':max_bin,
                'num_leaves':num_leaves,
                'random_state':random_state,
                'n_jobs':16
            },
            'fit':{
            },
        },
    }
    
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='tune 615 s3')
    val_mae_mean = np.mean(df_his.valid)
    val_mae_var = np.var(df_his.valid)
    train_mae_mean = np.mean(df_his.train)
    train_mae_var = np.var(df_his.train)
    
    trial.set_user_attr('val_mae', val_mae_mean)
    trial.set_user_attr('train_mae', train_mae_mean)
    trial.set_user_attr('mae_diff', val_mae_mean-train_mae_mean)
    trial.set_user_attr('val_mae_var', val_mae_var)

    return val_mae_mean

study = optuna.create_study()
study.optimize(objective, n_trials=200)

[I 2019-06-01 13:51:27,516] Finished a trial resulted in value: 1.8565777134251438. Current best value is 1.8565777134251438 with parameters: {'learning_rate': 0.26936113157977587, 'feature_fraction': 0.7087042760647086, 'bagging_fraction': 0.6393564866861171, 'min_data_in_leaf': 346, 'lambda_l1': 2.7128406989936945e-05, 'lambda_l2': 0.03342792963479392, 'max_bin': 43, 'num_leaves': 23, 'random_state': 2151}.
[I 2019-06-01 13:54:21,980] Finished a trial resulted in value: 1.870323991388399. Current best value is 1.8565777134251438 with parameters: {'learning_rate': 0.26936113157977587, 'feature_fraction': 0.7087042760647086, 'bagging_fraction': 0.6393564866861171, 'min_data_in_leaf': 346, 'lambda_l1': 2.7128406989936945e-05, 'lambda_l2': 0.03342792963479392, 'max_bin': 43, 'num_leaves': 23, 'random_state': 2151}.
[I 2019-06-01 13:56:25,924] Finished a trial resulted in value: 1.8893073751193867. Current best value is 1.8565777134251438 with parameters: {'learning_rate': 0.2693611315797

[I 2019-06-01 14:30:57,318] Finished a trial resulted in value: 1.8364178971105323. Current best value is 1.8364178971105323 with parameters: {'learning_rate': 0.4022521604373593, 'feature_fraction': 0.6612668453599558, 'bagging_fraction': 0.8723151172628785, 'min_data_in_leaf': 388, 'lambda_l1': 3.956488363350095, 'lambda_l2': 0.004115368908322646, 'max_bin': 44, 'num_leaves': 32, 'random_state': 309}.
[I 2019-06-01 14:34:00,020] Finished a trial resulted in value: 1.8528869087822206. Current best value is 1.8364178971105323 with parameters: {'learning_rate': 0.4022521604373593, 'feature_fraction': 0.6612668453599558, 'bagging_fraction': 0.8723151172628785, 'min_data_in_leaf': 388, 'lambda_l1': 3.956488363350095, 'lambda_l2': 0.004115368908322646, 'max_bin': 44, 'num_leaves': 32, 'random_state': 309}.
[I 2019-06-01 14:36:52,525] Finished a trial resulted in value: 1.8676908128326752. Current best value is 1.8364178971105323 with parameters: {'learning_rate': 0.4022521604373593, 'featu

[I 2019-06-01 15:16:18,654] Finished a trial resulted in value: 1.8381342932038829. Current best value is 1.8364178971105323 with parameters: {'learning_rate': 0.4022521604373593, 'feature_fraction': 0.6612668453599558, 'bagging_fraction': 0.8723151172628785, 'min_data_in_leaf': 388, 'lambda_l1': 3.956488363350095, 'lambda_l2': 0.004115368908322646, 'max_bin': 44, 'num_leaves': 32, 'random_state': 309}.
[I 2019-06-01 15:18:10,194] Finished a trial resulted in value: 1.8664765144414914. Current best value is 1.8364178971105323 with parameters: {'learning_rate': 0.4022521604373593, 'feature_fraction': 0.6612668453599558, 'bagging_fraction': 0.8723151172628785, 'min_data_in_leaf': 388, 'lambda_l1': 3.956488363350095, 'lambda_l2': 0.004115368908322646, 'max_bin': 44, 'num_leaves': 32, 'random_state': 309}.
[I 2019-06-01 15:20:50,352] Finished a trial resulted in value: 1.8599699279164401. Current best value is 1.8364178971105323 with parameters: {'learning_rate': 0.4022521604373593, 'featu

[I 2019-06-01 15:59:06,567] Finished a trial resulted in value: 1.8447495970712955. Current best value is 1.8364178971105323 with parameters: {'learning_rate': 0.4022521604373593, 'feature_fraction': 0.6612668453599558, 'bagging_fraction': 0.8723151172628785, 'min_data_in_leaf': 388, 'lambda_l1': 3.956488363350095, 'lambda_l2': 0.004115368908322646, 'max_bin': 44, 'num_leaves': 32, 'random_state': 309}.
[I 2019-06-01 16:00:34,860] Finished a trial resulted in value: 1.8743432380957064. Current best value is 1.8364178971105323 with parameters: {'learning_rate': 0.4022521604373593, 'feature_fraction': 0.6612668453599558, 'bagging_fraction': 0.8723151172628785, 'min_data_in_leaf': 388, 'lambda_l1': 3.956488363350095, 'lambda_l2': 0.004115368908322646, 'max_bin': 44, 'num_leaves': 32, 'random_state': 309}.
[I 2019-06-01 16:02:21,114] Finished a trial resulted in value: 1.905678632952266. Current best value is 1.8364178971105323 with parameters: {'learning_rate': 0.4022521604373593, 'featur

[I 2019-06-01 16:50:03,669] Finished a trial resulted in value: 1.8261509608453104. Current best value is 1.8261509608453104 with parameters: {'learning_rate': 0.2870492749454202, 'feature_fraction': 0.6517012091453672, 'bagging_fraction': 0.8323551618064362, 'min_data_in_leaf': 311, 'lambda_l1': 1.9427204810905014e-06, 'lambda_l2': 0.004778980652793753, 'max_bin': 63, 'num_leaves': 32, 'random_state': 9809}.
[I 2019-06-01 16:52:25,499] Finished a trial resulted in value: 1.9108851706610677. Current best value is 1.8261509608453104 with parameters: {'learning_rate': 0.2870492749454202, 'feature_fraction': 0.6517012091453672, 'bagging_fraction': 0.8323551618064362, 'min_data_in_leaf': 311, 'lambda_l1': 1.9427204810905014e-06, 'lambda_l2': 0.004778980652793753, 'max_bin': 63, 'num_leaves': 32, 'random_state': 9809}.
[I 2019-06-01 16:55:32,830] Finished a trial resulted in value: 1.849247647034251. Current best value is 1.8261509608453104 with parameters: {'learning_rate': 0.2870492749454

...

[I 2019-06-01 18:16:20,565] Finished a trial resulted in value: 1.95711700615869. Current best value is 1.8217157873112049 with parameters: {'learning_rate': 0.366101981751712, 'feature_fraction': 0.6408567868689926, 'bagging_fraction': 0.9270846520095388, 'min_data_in_leaf': 217, 'lambda_l1': 0.00039021162427586067, 'lambda_l2': 18.797153578941018, 'max_bin': 58, 'num_leaves': 32, 'random_state': 5386}.
[I 2019-06-01 18:17:49,051] Finished a trial resulted in value: 1.8773446830309124. Current best value is 1.8217157873112049 with parameters: {'learning_rate': 0.366101981751712, 'feature_fraction': 0.6408567868689926, 'bagging_fraction': 0.9270846520095388, 'min_data_in_leaf': 217, 'lambda_l1': 0.00039021162427586067, 'lambda_l2': 18.797153578941018, 'max_bin': 58, 'num_leaves': 32, 'random_state': 5386}.
[I 2019-06-01 18:20:07,752] Finished a trial resulted in value: 1.8524249967585957. Current best value is 1.8217157873112049 with parameters: {'learning_rate': 0.366101981751712, 'fe

[I 2019-06-01 19:03:27,156] Finished a trial resulted in value: 1.8499512865273173. Current best value is 1.8217157873112049 with parameters: {'learning_rate': 0.366101981751712, 'feature_fraction': 0.6408567868689926, 'bagging_fraction': 0.9270846520095388, 'min_data_in_leaf': 217, 'lambda_l1': 0.00039021162427586067, 'lambda_l2': 18.797153578941018, 'max_bin': 58, 'num_leaves': 32, 'random_state': 5386}.
[I 2019-06-01 19:06:07,881] Finished a trial resulted in value: 1.8451585059440063. Current best value is 1.8217157873112049 with parameters: {'learning_rate': 0.366101981751712, 'feature_fraction': 0.6408567868689926, 'bagging_fraction': 0.9270846520095388, 'min_data_in_leaf': 217, 'lambda_l1': 0.00039021162427586067, 'lambda_l2': 18.797153578941018, 'max_bin': 58, 'num_leaves': 32, 'random_state': 5386}.
[I 2019-06-01 19:08:54,827] Finished a trial resulted in value: 1.8363101988121346. Current best value is 1.8217157873112049 with parameters: {'learning_rate': 0.366101981751712, '

[I 2019-06-01 19:52:39,828] Finished a trial resulted in value: 1.8195786658499837. Current best value is 1.8195786658499837 with parameters: {'learning_rate': 0.31532839108246236, 'feature_fraction': 0.6949599136321848, 'bagging_fraction': 0.9158226087007694, 'min_data_in_leaf': 247, 'lambda_l1': 1.7429139152241337e-05, 'lambda_l2': 0.5636496844235925, 'max_bin': 87, 'num_leaves': 31, 'random_state': 5319}.
[I 2019-06-01 19:55:38,678] Finished a trial resulted in value: 1.8337009533864628. Current best value is 1.8195786658499837 with parameters: {'learning_rate': 0.31532839108246236, 'feature_fraction': 0.6949599136321848, 'bagging_fraction': 0.9158226087007694, 'min_data_in_leaf': 247, 'lambda_l1': 1.7429139152241337e-05, 'lambda_l2': 0.5636496844235925, 'max_bin': 87, 'num_leaves': 31, 'random_state': 5319}.
[I 2019-06-01 19:58:26,129] Finished a trial resulted in value: 1.831073723562586. Current best value is 1.8195786658499837 with parameters: {'learning_rate': 0.315328391082462

[I 2019-06-01 20:42:39,157] Finished a trial resulted in value: 1.851021284901659. Current best value is 1.8195786658499837 with parameters: {'learning_rate': 0.31532839108246236, 'feature_fraction': 0.6949599136321848, 'bagging_fraction': 0.9158226087007694, 'min_data_in_leaf': 247, 'lambda_l1': 1.7429139152241337e-05, 'lambda_l2': 0.5636496844235925, 'max_bin': 87, 'num_leaves': 31, 'random_state': 5319}.
[I 2019-06-01 20:45:31,096] Finished a trial resulted in value: 1.8452942774344159. Current best value is 1.8195786658499837 with parameters: {'learning_rate': 0.31532839108246236, 'feature_fraction': 0.6949599136321848, 'bagging_fraction': 0.9158226087007694, 'min_data_in_leaf': 247, 'lambda_l1': 1.7429139152241337e-05, 'lambda_l2': 0.5636496844235925, 'max_bin': 87, 'num_leaves': 31, 'random_state': 5319}.
[I 2019-06-01 20:47:33,965] Finished a trial resulted in value: 1.8561104404142543. Current best value is 1.8195786658499837 with parameters: {'learning_rate': 0.315328391082462

[I 2019-06-01 21:30:51,746] Finished a trial resulted in value: 1.8255032769417323. Current best value is 1.8195786658499837 with parameters: {'learning_rate': 0.31532839108246236, 'feature_fraction': 0.6949599136321848, 'bagging_fraction': 0.9158226087007694, 'min_data_in_leaf': 247, 'lambda_l1': 1.7429139152241337e-05, 'lambda_l2': 0.5636496844235925, 'max_bin': 87, 'num_leaves': 31, 'random_state': 5319}.
[I 2019-06-01 21:32:19,886] Finished a trial resulted in value: 1.8872701369782456. Current best value is 1.8195786658499837 with parameters: {'learning_rate': 0.31532839108246236, 'feature_fraction': 0.6949599136321848, 'bagging_fraction': 0.9158226087007694, 'min_data_in_leaf': 247, 'lambda_l1': 1.7429139152241337e-05, 'lambda_l2': 0.5636496844235925, 'max_bin': 87, 'num_leaves': 31, 'random_state': 5319}.
[I 2019-06-01 21:34:54,130] Finished a trial resulted in value: 1.8693249707961541. Current best value is 1.8195786658499837 with parameters: {'learning_rate': 0.31532839108246

In [25]:
for trial_i in mytrial:
    db.insert(trial_i)
df_trial = db.select()
df_trial['kfold'] = df_trial['param'].apply(lambda x: x['kfold'])
df_trial['algorithm-init'] = df_trial['param'].apply(lambda x: x['algorithm']['init'])

In [26]:
df_trial[(df_trial['remark']=='tune 615 s3')][['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff','algorithm-init','kfold']].sort_values(by=['val_mae'])

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff,algorithm-init,kfold
1011,2019-06-01 19:52:39.785926,tune 615 s3,26,1.469417,4.083343e-05,1.819579,0.000027,0.350162,"{'learning_rate': 0.31532839108246236, 'feature_fraction': 0.6949599136321848, 'bagging_fraction': 0.9158226087007694, 'min_data_in_leaf': 247, 'lambda_l1': 1.7429139152241337e-05, 'lambda_l2': 0.5636496844235925, 'max_bin': 87, 'num_leaves': 31, 'random_state': 5319, 'n_jobs': 16}","{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}"
955,2019-06-01 17:41:15.859140,tune 615 s3,26,1.445090,3.068113e-05,1.821716,0.000016,0.376625,"{'learning_rate': 0.366101981751712, 'feature_fraction': 0.6408567868689926, 'bagging_fraction': 0.9270846520095388, 'min_data_in_leaf': 217, 'lambda_l1': 0.00039021162427586067, 'lambda_l2': 18.797153578941018, 'max_bin': 58, 'num_leaves': 32, 'random_state': 5386, 'n_jobs': 16}","{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}"
1016,2019-06-01 20:06:39.154664,tune 615 s3,26,1.447371,1.005891e-05,1.822511,0.000085,0.375140,"{'learning_rate': 0.3548027345464713, 'feature_fraction': 0.7237606340801555, 'bagging_fraction': 0.888233421591839, 'min_data_in_leaf': 303, 'lambda_l1': 0.0001107345386750328, 'lambda_l2': 2.8526044882449812e-05, 'max_bin': 79, 'num_leaves': 32, 'random_state': 2320, 'n_jobs': 16}","{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}"
1021,2019-06-01 20:20:51.704530,tune 615 s3,26,1.426234,7.935431e-05,1.822705,0.000155,0.396471,"{'learning_rate': 0.3609959404258411, 'feature_fraction': 0.6943441416245035, 'bagging_fraction': 0.9339904711694377, 'min_data_in_leaf': 219, 'lambda_l1': 0.0008493943691639534, 'lambda_l2': 4.883015731817629e-06, 'max_bin': 100, 'num_leaves': 31, 'random_state': 1514, 'n_jobs': 16}","{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}"
954,2019-06-01 17:38:10.765511,tune 615 s3,26,1.429808,2.969584e-05,1.824189,0.000093,0.394380,"{'learning_rate': 0.3671831540069572, 'feature_fraction': 0.641758738206807, 'bagging_fraction': 0.9356216641308657, 'min_data_in_leaf': 223, 'lambda_l1': 0.0011020313754484974, 'lambda_l2': 9.48312025978774, 'max_bin': 72, 'num_leaves': 32, 'random_state': 5200, 'n_jobs': 16}","{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}"
1000,2019-06-01 19:23:24.639481,tune 615 s3,26,1.524423,4.820107e-06,1.824671,0.000113,0.300248,"{'learning_rate': 0.26073664764091997, 'feature_fraction': 0.6405959935459784, 'bagging_fraction': 0.8738995381137459, 'min_data_in_leaf': 238, 'lambda_l1': 1.4985556014012036e-06, 'lambda_l2': 0.0014641456757163696, 'max_bin': 63, 'num_leaves': 32, 'random_state': 4561, 'n_jobs': 16}","{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}"
1024,2019-06-01 20:28:17.427837,tune 615 s3,26,1.439991,5.522671e-06,1.825048,0.000027,0.385057,"{'learning_rate': 0.3559404402875731, 'feature_fraction': 0.8254368183238012, 'bagging_fraction': 0.9572166912379101, 'min_data_in_leaf': 301, 'lambda_l1': 0.00010528682552497587, 'lambda_l2': 1.2667580321954649e-05, 'max_bin': 100, 'num_leaves': 31, 'random_state': 1441, 'n_jobs': 16}","{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}"
953,2019-06-01 17:35:05.739201,tune 615 s3,26,1.511547,1.175223e-07,1.825382,0.000098,0.313835,"{'learning_rate': 0.2641804493763099, 'feature_fraction': 0.6247617096240117, 'bagging_fraction': 0.7680877757063254, 'min_data_in_leaf': 242, 'lambda_l1': 0.0013908309282393686, 'lambda_l2': 0.3406084549313588, 'max_bin': 82, 'num_leaves': 32, 'random_state': 5364, 'n_jobs': 16}","{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}"
1049,2019-06-01 21:30:51.718966,tune 615 s3,26,1.411248,4.135785e-06,1.825503,0.000053,0.414255,"{'learning_rate': 0.4203784894245139, 'feature_fraction': 0.7020848458301066, 'bagging_fraction': 0.9434028821272483, 'min_d

In [28]:
db.commit()